# Visualization using maps in Python


Another perfect tool for vizualisation data processed in Python is `folium`. It builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js library. It manipulate your data in Python, then visualize it in on a Leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing Vincent/Vega visualizations as markers on the map.

In [ ]:
%matplotlib inline

In [ ]:
!sudo -H pip3 install -U folium
!sudo -H pip3 install -U geopandas

`folium` provides very detailed map and we may use it to visualize geodata localized in a small scale. Let's get the data from the [Citibike API](http://www.citibikenyc.com/stations/json):

In [ ]:
import requests
import pandas as pd

url = 'http://www.citibikenyc.com/stations/json'
results = requests.get(url).json()
data = results["stationBeanList"]

citibike = pd.DataFrame(data)
citibike.set_index('id', inplace=True)

citibike

#### Selecting tiles

The first step is to create a map. At the very basic level, we select the location, zoom level, and potentially the tiles (i.e., the style of the map) for the background. The default is 'OpenStreetMap', but often for visualizations we prefer other, more visually neutral styles. (See http://folium.readthedocs.io/en/latest/quickstart.html for more tile optionS)

In [ ]:
import folium
fmap = folium.Map(location=[40.73, -74], zoom_start=13, tiles='OpenStreetMap')
fmap

In [ ]:
import folium
fmap = folium.Map(location=[40.73, -74], zoom_start=12,  tiles='cartodbpositron')
fmap

#### Adding Markers 

For every station, we are going to add a marker in the map:
* Using the longitude and latitude for the location 
* Modify the color of the marker to reflect the status of the station
* Modify the opacity to be the percentage of bikes in the station. 
* Modify the size of the circle to corresponds to the size of the station.

In [ ]:
for name, row in citibike.iterrows():
    
    # Define the opacity of the marker to be proportional to the percentage of bikes in the station
    opacity = row["availableBikes"]/row["totalDocks"] if row["statusValue"] == 'In Service' else 1.0
    # Make the color green for the working stations, red otherwise
    color = "green" if row["statusValue"] == 'In Service' else "red"
    # The size of the marker is proportional to the number of docks
    size = row["totalDocks"]/10 if row["statusValue"] == 'In Service' else 5

    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                        radius = size,
                        color='black', weight=0.5, 
                        fill=True,
                        fill_opacity = opacity,
                        fill_color = color,
                       ).add_to(fmap)
    


In [ ]:
fmap

#### Adding popups to the markers

For each marker, we can also have a popup with text, html, or even other charts/visualizations. Here is an example of adding an HTML popup to each marker.

In [ ]:
fmap = folium.Map(location=[40.73, -74], zoom_start=13,  tiles='cartodbpositron')

for name, row in citibike.iterrows():
    
    # Define the opacity of the marker to be proportional to the percentage of bikes in the station
    opacity = row["availableBikes"]/row["totalDocks"] if row["statusValue"] == 'In Service' else 1.0
    # Make the color green for the working stations, red otherwise
    color = "green" if row["statusValue"] == 'In Service' else "red"
    # The size of the marker is proportional to the number of docks
    size = row["totalDocks"]/5 if row["statusValue"] == 'In Service' else 5
    
   
    # The code below defines a pop-up for each station with details such as 
    # the address, number of bikes, capacity, etc.
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Address: </strong>" + row["stAddress1"] + \
           "<br><strong>Available Bikes: </strong>" + str(row["availableBikes"]) + \
            "<br><strong>Total Docks: </strong>" + str(row["totalDocks"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)

    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                        radius = size,
                        popup = popup, 
                        color='black', weight=0.5, 
                        fill=True,
                        fill_opacity = opacity,
                        fill_color = color,
                       ).add_to(fmap)

In [ ]:
    
fmap

In [ ]:
# This code below is for generating an image screenshot from a Folium map
# It is kind of convoluted, as it involves saving an HTML file
# and then launching a Selenium-driven browser and saving a screenshot
# NOTE: This requires having a properly working installation
# of Selenium


import os
import time

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1600, 1600))
display.start()

delay=5
fn='citibike.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
fmap.save(fn)

In [ ]:
!sudo pip3 install pyvirtualdisplay

In [ ]:
from selenium import webdriver
browser = webdriver.Firefox()
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('citibike.png')
browser.quit()

#### Using shapefiles 

In [ ]:
# Dataset from NYC Open Data: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq
!curl 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=GeoJSON' -o data/nyc-neighborhoods.geojson

In [ ]:
# NYC Zipcodes
!curl 'https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip' -o 'data/ZIP_CODE_040114.zip'

In [ ]:
import geopandas as gpd

nyc_neighborhoods = gpd.GeoDataFrame.from_file('data/nyc-neighborhoods.geojson')
nyc_neighborhoods = nyc_neighborhoods[['ntacode', 'ntaname', 'geometry']]
nyc_neighborhoods.head(10)

In [ ]:
nyc_neighborhoods.plot(
    figsize=(20,20), 
    color = 'white', 
    edgecolor = 'black'
)

In [ ]:
import folium
fmap = folium.Map(location=[40.73, -74], zoom_start=12, tiles='cartodbpositron')

folium.GeoJson(nyc_neighborhoods,
               name='NYC Neighborhoods',
               style_function=lambda feature: {
                    'fillColor': '#c0fefe',
                    'color': 'black',
                    'weight': 1,
                    'fillOpacity': 0.25
                }
              ).add_to(fmap)

fmap

We first transform the Citibike Dataframe into a GeoPandas datagframe, by creating 
a column that contains Points, naming the column "geometry" and then 
setting the CRS (coordinate system) to be the same as the one for NYC neighborhoods

In [ ]:
from shapely.geometry import Point

citibike['geometry'] = citibike.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1 )
citibike_gs = gpd.GeoDataFrame.from_records(citibike)


#### Spatial Join using GeoPandas

Now we join the `nyc_neighborhoods` dataframe that describes the NYC neighborhoods with the `citibike_gs` that has the locations of the Citibike stations.

In [ ]:
citibike_gs.crs = nyc_neighborhoods.crs
stations_to_neighborhoods = gpd.sjoin(nyc_neighborhoods, citibike_gs, how="inner", op='intersects')

In [ ]:
totaldocks = pd.pivot_table(
    data = stations_to_neighborhoods, 
    index = 'ntacode',
    values = 'totalDocks', 
    aggfunc = 'sum'
).drop(['MN99', 'BK99']) # drop the 'misc' areas

totaldocks.head(5)

In [ ]:
# GeoPandas choropleths / not very appealing visually
# 
# nyc.set_index('ntacode').join(totaldocks, how='left').fillna(0).plot(
#     figsize=(20,20), column='totalDocks', cmap='OrRd', scheme='Quantiles', linewidth=0.1)

In [ ]:
fmap = folium.Map(location=[40.73, -74], zoom_start=12, tiles='cartodbpositron')
# folium.LayerControl().add_to(fmap)

fmap.choropleth(geo_data='data/nyc-neighborhoods.geojson', 
                data=totaldocks.reset_index(),
                columns=['ntacode', 'totalDocks'],
                key_on='feature.properties.ntacode',
                fill_color='OrRd', 
                fill_opacity=0.5, 
                line_opacity=0.1,
                legend_name='Total Docks'
               )
#folium.LayerControl().add_to(fmap)
fmap